In [ ]:
#Hello Danika, I created this extra folder for all the paper regarding tests. 
#I also copied the relevant drivers and wrappers into this folder since I did not want to mess with the original files in case I need to change something.
#I also changed the code in the way that it creates a file with all the raw data and adds the data immediately to the file.



In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import numpy as np
import time
import csv
from Dispensing_wrapper_robot import *
from scale_driver import *
from motor_driver import *


In [ ]:
coms = {
    'scaleCom': 'COM7',
    'motorsCom': 'COM9',
}

#Robot Connection
robot = Robot()
robot.initialize()
robot.GoTo_InitialPoint()

In [ ]:
robot.GoTo_InitialPoint()

In [62]:
#Motor Connection
motors = SerialConnection(coms['motorsCom'], 9600, 10)
dispenser_motor = Motor(motors, 0)

#Scale Connection
scale = Scale(coms['scaleCom'], 9600, 10)
#scale.connect() -> this will happen directly in the calibartion loop 
#scale.tare()

In [65]:
# Define your parameters
acceleration = 10
speed = 5
material = "SuperP"
version_inside = "Design4"
version_outside = "Design1"

In [64]:
dispenser_motor.move(4000, True) 
#I ran some random tests and wrote some numbers down to get a feeling -> not looking good to be honest, i think with Super P we need a lot of turns to get a reproducibele outcome
#0.0063g per 200 steps
#0,0086 per 400
#0,0026 per 200
#0,0083 per 200
#0,0167 per 1000
#0,0172 per 1000
#0,0132 per 1000


In [66]:
#Error test: Test with Super P and Design 4, see at which point the error does not change anymore significantly 
#Motor is doing 200steps for one full turn, Design 4 has 12 pits which means 200/12 = 16.6 steps are neccesary to empty one pit
#Calibration step: 200/n, 2x200/n, 3x200/n, 4x200/n,...till teh first hole is being emptied (e.g 12 holes for design 4 so we empty six holes so the first hole which was filled gets emptied again) + 200, 1000,...

calibration = Calibration(acceleration, speed, material, version_inside, version_outside, dispenser_motor, scale, robot)

#calibration_steps = [17, 34, 51, 68, 85, 102, 200, 400, 600, 800, 1000, 2000] 200/12 = 16.6, so 17 steps, we could do factors of 17 like shown here
calibration_steps = [17, 33, 50, 67, 83, 100, 200, 1000, 2000, 3000, 4000] # or we take factors of 16.6 and round up or down to the nearest integer liek shown here
calibration_repeat = 10

calibration.calibrate(calibration_steps, calibration_repeat, "Vial1")

calibration.save_calibration()

Repeat: 1
Scale opened
Step: 17, Weight: -0.0003
Scale closed
Scale opened
Step: 33, Weight: -0.0001
Scale closed
Scale opened
Step: 50, Weight: 0.0
Scale closed
Scale opened
Step: 67, Weight: 0.0
Scale closed
Scale opened
Step: 83, Weight: 0.0
Scale closed
Scale opened
Step: 100, Weight: -0.0001
Scale closed
Scale opened
Step: 200, Weight: 0.0
Scale closed
Scale opened
Step: 1000, Weight: 0.0001
Scale closed
Scale opened
Step: 2000, Weight: 0.0006
Scale closed
Scale opened
Step: 3000, Weight: 0.0005
Scale closed
Scale opened
Step: 4000, Weight: 0.0004
Scale closed
Repeat: 2
Scale opened
Step: 17, Weight: 0.0
Scale closed
Scale opened
Step: 33, Weight: 0.0
Scale closed
Scale opened
Step: 50, Weight: 0.0
Scale closed
Scale opened
Step: 67, Weight: 0.0
Scale closed
Scale opened
Step: 83, Weight: 0.0
Scale closed
Scale opened
Step: 100, Weight: 0.0002
Scale closed
Scale opened
Step: 200, Weight: 0.0
Scale closed
Scale opened
Step: 1000, Weight: 0.0
Scale closed
Scale opened
Step: 2000, We

c:\Users\digic\OneDrive\Desktop\Slurry Formation 2024\SlurryBot\Slurry_Bot\PaperTest\Dispensing_wrapper_robot.py:95: RuntimeWarning: divide by zero encountered in divide
  self.relative_std_error_weights = self.std_error_weights / self.avg_weights
c:\Users\digic\OneDrive\Desktop\Slurry Formation 2024\SlurryBot\Slurry_Bot\PaperTest\Dispensing_wrapper_robot.py:95: RuntimeWarning: invalid value encountered in divide
  self.relative_std_error_weights = self.std_error_weights / self.avg_weights


[SDK][ERROR][2024-05-28 16:58:53][base.py:293] - - [main-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host
[SDK][ERROR][2024-05-28 16:58:53][base.py:247] - - [report-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host


In [68]:
calibration_steps = [17, 33, 50, 67, 83, 100, 200, 1000, 2000, 3000, 4000]
calibration_repeat = 10

material_per_step = 0.00005
total_steps = sum(calibration_steps)*calibration_repeat
total_material= sum(step*material_per_step for step in calibration_steps)*calibration_repeat
print(f'Total material used for calibration: {total_material}g')
print(f'Total number of steps: {total_steps}')

Total material used for calibration: 5.2749999999999995g
Total number of steps: 105500
